In [55]:
import os
import pandas as pd
import numpy as np
from pprint import pprint
from pathlib import Path
from collections import Counter
import pickle
import random
import argparse
import time
from datetime import datetime

# torch:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

from pytorch_lightning import LightningDataModule, LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report
from sklearn.model_selection import train_test_split

from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor
from transformers import BertTokenizer, BertConfig, BertModel,XLMTokenizer, XLMModel
from prompts import assesmentPrompt_template, Instruction_templates, Psychology_template,\
    Sensitive_replace_dict, generate_psychology_prompt
class Arg:
    version = 1
    # data
    epochs: int = 5  # Max Epochs, BERT paper setting [3,4,5]
    max_length: int = 350  # Max Length input size
    report_cycle: int = 30  # Report (Train Metrics) Cycle
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    a_hidden_size = 0 # BERT-base: 768, BERT-large: 1024, BERT paper setting
    t_hidden_size = 768
    t_x_hidden_size = a_hidden_size+t_hidden_size
    batch_size: int = 8
            
class BertPooler(nn.Module):
    def __init__(self,hidden_size):
        super().__init__()
        self.dense = nn.Linear(hidden_size, hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output
    
    
class Model(LightningModule):
    def __init__(self, args,config):
        super().__init__()
        # config:
        
        self.args = args
        self.config = config
        self.batch_size = self.args.batch_size
        
        # meta data:
        self.epochs_index = 0
        self.label_cols = 'dementia_labels'
        self.label_names = ['Control','ProbableAD']
        self.num_labels = 2
        self.t_embed_type = self.config['t_embed']
        self.a_embed_type = self.config['a_embed']
        self.a_hidden = self.args.a_hidden_size
        
        # --> HERE STEP 1 <--
        # ATTRIBUTES TO SAVE BATCH OUTPUTS
        self.test_step_outputs = []   # save outputs in each batch to compute metric overall epoch
        self.test_step_targets = []   # save targets in each batch to compute metric overall epoch
        self.val_step_outputs = []        # save outputs in each batch to compute metric overall epoch
        self.val_step_targets = []        # save targets in each batch to compute metric overall epoch


        if self.t_embed_type == "mbert":
            self.t_hidden = self.args.t_hidden_size
            
            t_pretrained = 'bert-base-multilingual-uncased'
            self.t_tokenizer = BertTokenizer.from_pretrained(t_pretrained)
            self.t_model = BertModel.from_pretrained(t_pretrained)
            
            
        elif self.t_embed_type == "xlm":
            self.t_hidden = self.args.t_x_hidden_size
            
            t_pretrained = 'xlm-mlm-100-1280'
            self.t_tokenizer = XLMTokenizer.from_pretrained(t_pretrained)
            self.t_model = XLMModel.from_pretrained(t_pretrained)
            self.pooler = BertPooler(self.t_hidden)
            
        self.hidden = int(self.a_hidden + self.t_hidden)
        
        if self.a_embed_type == "en":
            a_pretrained =  "jonatasgrosman/wav2vec2-large-xlsr-53-english"
            
        elif self.a_embed_type == "gr":
            a_pretrained =  "lighteternal/wav2vec2-large-xlsr-53-greek"

        elif self.a_embed_type == "multi":
            a_pretrained = "voidful/wav2vec2-xlsr-multilingual-56"
            
        elif self.a_embed_type == "wv":
            a_pretrained ='facebook/wav2vec2-base'
            
        self.a_tokenizer = Wav2Vec2FeatureExtractor.from_pretrained(a_pretrained)
        self.a_model = Wav2Vec2Model.from_pretrained(a_pretrained)
        
        
        self.clf1 = nn.Linear(self.hidden, int(self.hidden/2))
        self.clf2 = nn.Linear(int(self.hidden/2), self.num_labels)
        
            
            
    # def forward(self, text, audio):
    def forward(self, text):
        
        if self.t_embed_type == "mbert":
            t_out = self.t_model(text)[1] 

            
        elif self.t_embed_type == "xlm":
            t_out = self.t_model(text)[0]
            t_out = self.pooler(t_out)
            
            
        # a_out = self.a_model(audio)['extract_features']#[2] #last_hidden_state , feature extraction
        # a_out = a_out[:, 0, :] 
        
        #print(a_out)
        #print(a_out['extract_features'].shape) # ([8, 437, 512])
        #print(a_out['last_hidden_state'].shape) # ([8, 437, 1024]) => pooling 필요
        
        
        # output = torch.cat((t_out,a_out),axis=1)   
        output = t_out
        #print(output.shape)
        
        logits = self.clf2(self.clf1(output))
    
        return logits
        

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config['lr'])
        scheduler = ExponentialLR(optimizer, gamma=0.5)
        
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def preprocess_dataframe(self):
        
        tg_sr = 16000
        t_col_name = "text" 
        # a_col_name = "path"         
        # df = pd.read_json('/mnt/Internal/FedASR/Data/230126_total_asr_data.json')
        df = pd.read_csv(config.file_in)

        def Augment_info_df(df_test):
            # 將 'path' 欄位進行字串操作
            df_test['path'] = df_test['path'].str.rstrip('.wav')

            # 使用 str.split 拆分 'path' 欄位
            df_test[['session', 'role', 'number', 'start_time', 'end_time']] = df_test['path'].str.split('_', expand=True)

            # 如果 'number' 欄位的末尾包含 '.wav'，進行一次額外的拆分
            df_test['number'] = df_test['number'].str.rstrip('.wav')

            # 將 'start_time' 和 'end_time' 欄位轉換為數值型別
            df_test[['start_time', 'end_time']] = df_test[['start_time', 'end_time']].astype(int)

            return df_test
        df = Augment_info_df(df)


        # Packer
        def Packer(df_test) -> dict:
            People_dict = dict(tuple(df_test.groupby('session')))
            return People_dict

        def Dialogueturn2corpus(data_frame, mode='text'): #mode can be 'pred_str' or 'text'
            # 按 'start_time' 列進行排序
            sorted_data_frame = data_frame.sort_values(by='start_time')

            # 添加前綴並使用 '\n' 進行拼接
            processed_text = sorted_data_frame.apply(lambda row: f"{row['role']}: {row[mode]}", axis=1).str.cat(sep='\n')

            return processed_text

        def filter_people_dict(People_dict, mode="INV+PAR", verbose=False) -> dict:# 'INV' , 'PAR', 'INV+PAR'
            filtered_people_dict = {}

            for session, data_frame in People_dict.items():
                # 使用 query 過濾 'role' 為 'INV' 或 'PAR'
                if mode == "PAR":
                    filtered_data_frame = data_frame.query("role == 'PAR'")
                    filtered_people_dict[session] = filtered_data_frame
                elif mode == "INV":
                    filtered_data_frame = data_frame.query("role == 'INV'")
                    filtered_people_dict[session] = filtered_data_frame
                elif mode == "INV+PAR":
                    filtered_people_dict[session] = data_frame
                else:
                    raise OSError
            
            if verbose:
                # 印出過濾後的 People_dict 中每個 session 的 DataFrame
                for session, data_frame in filtered_people_dict.items():
                    print(f"Session: {session}")
                    print(data_frame)
                    print("\n")

            return filtered_people_dict

        # Dialogue Formatter
        def Dialogue_Formatter(People_dict, sep="\n",role_mode='PAR')->dict:
            session_df=pd.DataFrame()
            for session, data_frame in People_dict.items():
                if len(data_frame)>0:
                    total_info=data_frame.iloc[0].copy()
                    sessional_text = Dialogueturn2corpus(data_frame,mode='text')
                    sessional_predStr = Dialogueturn2corpus(data_frame,mode='pred_str')
                    
                    # total_info,'text']=sessional_text
                    # session_df.loc[session,'pred_str']=sessional_predStr
                    # session_df.loc[session,'role']=role_mode
                    # session_df.loc[session,'start_time']=data_frame['start_time'].min()
                    # session_df.loc[session,'end_time']=data_frame['end_time'].max()

                    total_info['text']=sessional_text
                    total_info['pred_str']=sessional_predStr
                    total_info['role']=role_mode
                    total_info['start_time']=data_frame['start_time'].min()
                    total_info['end_time']=data_frame['end_time'].max()
                    session_df = pd.concat([session_df, pd.DataFrame([total_info], index=[session])])
                else:
                    print(f"Session {session} has no data")
            return session_df
        df_train = df[df['ex'] == 'train']
        df_val = df[df['ex'] == 'dev']
        df_test = df[df['ex'] == 'test']

        def SentenceLvldf2SessionLvldf(df, role_mode="PAR"):
            People_dict=Packer(df)
            People_dict = filter_people_dict(People_dict, mode=role_mode, verbose=False)
            df_dialogue=Dialogue_Formatter(People_dict,role_mode)
            return df_dialogue

        df_train=SentenceLvldf2SessionLvldf(df_train)
        df_val=SentenceLvldf2SessionLvldf(df_val)
        df_test=SentenceLvldf2SessionLvldf(df_test)

        def Tokenize(df_data):
            df_data[t_col_name] = df_data[t_col_name].map(lambda x: self.t_tokenizer.encode(
                str(x),
                padding = 'max_length',
                max_length=self.args.max_length,
                truncation=True,
                ))
            return df_data
        df_train=Tokenize(df_train)
        df_val=Tokenize(df_val)
        df_test=Tokenize(df_test)
        df_test = df_test.reset_index(drop=True)

        # audio_root="/mnt/Internal/FedASR/Data/ADReSS-IS2020-data/clips"
        # # 원래 길이: 562992, batch 16: 90000, batch 8: 140000
        # # max_length=16000, truncation=True 이건 일단 돌려보고 결정 => 뒤쪽, 앞에쪽 뭐보면 좋을 지 그런거 check하면 좋으니까! 
        # df[a_col_name] = df[a_col_name].map(lambda x: self.a_tokenizer(
        #     f"{audio_root}/{x}",
        #     sampling_rate = tg_sr,
        #     max_length=100000, 
        #     truncation=True
        #     )['input_values'][0])

        self.train_data = TensorDataset(
            torch.tensor(df_train[t_col_name].tolist(), dtype=torch.long),
            # torch.tensor(df_train[a_col_name].tolist(), dtype=torch.float),
            torch.tensor(df_train[self.label_cols].tolist(), dtype=torch.long),
        )
        
        self.val_data = TensorDataset(
             torch.tensor(df_val[t_col_name].tolist(), dtype=torch.long),
            #  torch.tensor(df_val[a_col_name].tolist(), dtype=torch.float),
            torch.tensor(df_val[self.label_cols].tolist(), dtype=torch.long),
        )

        self.test_data = TensorDataset(
             torch.tensor(df_test[t_col_name].tolist(), dtype=torch.long),
            #  torch.tensor(df_test[a_col_name].tolist(), dtype=torch.float),
            torch.tensor(df_test[self.label_cols].tolist(), dtype=torch.long),
             torch.tensor(df_test.index.tolist(), dtype=torch.long),
        )

    def train_dataloader(self):
        
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.args.cpu_workers,
        )
    
    def val_dataloader(self):

        return DataLoader(
            self.val_data,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )
    
    def test_dataloader(self):

        return DataLoader(
            self.test_data,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )
    
    def training_step(self, batch, batch_idx):
        # token, audio, labels = batch  
        token,  labels = batch  
        # logits = self(token, audio) 
        logits = self(token) 
        loss = nn.CrossEntropyLoss()(logits, labels)   
        
        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        # token, audio, labels = batch  
        token, labels = batch  
        # logits = self(token, audio) 
        logits = self(token) 
        loss = nn.CrossEntropyLoss()(logits, labels)     
        
        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        # --> HERE STEP 2 <--
        self.val_step_outputs.append({
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        })
        # self.val_step_targets.append(y_true)
        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }
        
            
    def test_step(self, batch, batch_idx):
        # token, audio, labels,id_ = batch 
        token, labels,id_ = batch 
        print('id', id_)
        # logits = self(token, audio) 
        logits = self(token) 
        
        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        # --> HERE STEP 2 <--
        self.test_step_outputs.append({
            'y_true': y_true,
            'y_pred': y_pred,
        })
        # self.test_step_targets.append(y_true)
        return {
            'y_true': y_true,
            'y_pred': y_pred,
        }
    
    def on_validation_epoch_end(self):
        loss = torch.tensor(0, dtype=torch.float)
        # print("Value= ",self.val_step_outputs)
        # print("type(self.val_step_outputs)=",type(self.val_step_outputs))
        # print("type(self.val_step_outputs[0])=",type(self.val_step_outputs[0]))
        # print("type(self.val_step_outputs[0] loss)=",type(self.val_step_outputs[0]['loss']))
        for i in self.val_step_outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(self.val_step_outputs)
        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in self.val_step_outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']
            
        y_pred = np.asanyarray(y_pred)#y_temp_pred y_pred
        y_true = np.asanyarray(y_true)
        
        pred_dict = {}
        pred_dict['y_pred']= y_pred
        pred_dict['y_true']= y_true
        
        val_acc = accuracy_score(y_true=y_true, y_pred=y_pred)
        
        self.log("val_acc", val_acc)
        # print("y_pred= ", y_pred)
        # print('\n\n\n')
        # print("y_true= ", y_true)
        # print('\n\n\n')
        print("-------val_report-------")
        metrics_dict = classification_report(y_true, y_pred,zero_division=1,
                                             target_names = self.label_names, 
                                             output_dict=True)
        df_result = pd.DataFrame(metrics_dict).transpose()
        pprint(df_result)
        
        Output_dir=f"{SaveRoot}/result"
        os.makedirs(Output_dir, exist_ok=True)
        df_result.to_csv(
            f'{Output_dir}/{datetime.now().__format__("%m%d_%H%M")}_DM_MM_{self.t_embed_type}_{self.a_embed_type}_val.csv')

        pred_df = pd.DataFrame(pred_dict)
        pred_df.to_csv(
            f'{Output_dir}/{datetime.now().__format__("%m%d_%H%M")}_DM_MM_{self.t_embed_type}_{self.a_embed_type}_val_pred.csv')
        self.val_step_outputs.clear()
        # self.val_step_targets.clear()
        return {'loss': _loss}

    def on_test_epoch_end(self):

        y_true = []
        y_pred = []

        for i in self.test_step_outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']
            
        y_pred = np.asanyarray(y_pred)#y_temp_pred y_pred
        y_true = np.asanyarray(y_true)
        
        pred_dict = {}
        pred_dict['y_pred']= y_pred
        pred_dict['y_true']= y_true
        
        
        print("-------test_report-------")
        metrics_dict = classification_report(y_true, y_pred,zero_division=1,
                                             target_names = self.label_names, 
                                             output_dict=True)
        df_result = pd.DataFrame(metrics_dict).transpose()
        self.test_step_outputs.clear()
        # self.test_step_targets.clear()
        pprint(df_result)
        
        Output_dir=f"{SaveRoot}/result"
        os.makedirs(Output_dir, exist_ok=True)
        df_result.to_csv(
            f'{Output_dir}/{datetime.now().__format__("%m%d_%H%M")}_DM_MM_{self.t_embed_type}_{self.a_embed_type}_test.csv')

        pred_df = pd.DataFrame(pred_dict)
        pred_df.to_csv(
            f'{Output_dir}/{datetime.now().__format__("%m%d_%H%M")}_DM_MM_{self.t_embed_type}_{self.a_embed_type}_test_pred.csv')

    
def main(args,config):
    print("Using PyTorch Ver", torch.__version__)
    print("Fix Seed:", config['random_seed'])
    seed_everything( config['random_seed'])
        
    model = Model(args,config) 
    model.preprocess_dataframe()

    early_stop_callback = EarlyStopping(
        monitor='val_acc',
        patience=10,
        verbose=True,
        mode='max'
    )
    
    checkpoint_callback = ModelCheckpoint(
        dirpath=f"{SaveRoot}/Model/checkpoints",
        monitor='val_acc',
        auto_insert_metric_name=True,
        verbose=True,
        mode='max', 
        save_top_k=1,
      )    

    print(":: Start Training ::")
    #     
    trainer = Trainer(
        logger=False,
        callbacks=[early_stop_callback,checkpoint_callback],
        enable_checkpointing = True,
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        deterministic=True, # ensure full reproducibility from run to run you need to set seeds for pseudo-random generators,
        # For GPU Setup
        # gpus=[config['gpu']] if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32
    )
    trainer.fit(model)
    trainer.test(model,dataloaders=model.test_dataloader(),ckpt_path="best")
    
if __name__ == '__main__': 

    parser = argparse.ArgumentParser("main.py", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument("--gpu", type=int, default=1)
    parser.add_argument("--epochs", type=int, default=5)
    parser.add_argument("--lr", type=float, default=2e-5, help="learning rate")
    parser.add_argument("--random_seed", type=int, default=2023) 
    parser.add_argument("--t_embed", type=str, default="mbert") 
    parser.add_argument("--a_embed", type=str, default="en") 
    parser.add_argument("--SaveRoot", type=str, default='/mnt/External/Seagate/FedASR/LLaMa2/dacs') 
    parser.add_argument("--file_in", type=str, default='/home/FedASR/dacs/centralized/saves/results/data2vec-audio-large-960h_total.csv') 
    
    
    config = parser.parse_args(args=[])
    SaveRoot=config.SaveRoot
    
    print(config)
    args = Arg()
    args.epochs=config.epochs
    main(args,config.__dict__)       


"""

python 0207_DM_multi.py --gpu 1 --t_embed mbert --a_embed en
python 0207_DM_multi.py --gpu 1 --t_embed xlm --a_embed en

# don
python 0207_DM_multi.py --gpu 0 --t_embed xlm --a_embed gr
python 0207_DM_multi.py --gpu 1 --t_embed mbert --a_embed gr

"""

Seed set to 2023


Namespace(gpu=1, epochs=5, lr=2e-05, random_seed=2023, t_embed='mbert', a_embed='en', SaveRoot='/mnt/External/Seagate/FedASR/LLaMa2/dacs', file_in='/home/FedASR/dacs/centralized/saves/results/data2vec-audio-large-960h_total.csv')
Using PyTorch Ver 2.1.1+cu121
Fix Seed: 2023


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Session S009 has no data
Session S011 has no data
Session S017 has no data
Session S027 has no data
Session S029 has no data
Session S052 has no data
Session S080 has no data
Session S084 has no data
Session S086 has no data
Session S118 has no data
Session S124 has no data
Session S138 has no data
Session S145 has no data


ValueError: too many dimensions 'str'

In [59]:
args = Arg()
model = Model(args,config.__dict__) 
self=model
tg_sr = 16000
t_col_name = "text" 
# a_col_name = "path"         
# df = pd.read_json('/mnt/Internal/FedASR/Data/230126_total_asr_data.json')
df = pd.read_csv(config.file_in)

def Augment_info_df(df_test):
    # 將 'path' 欄位進行字串操作
    df_test['path'] = df_test['path'].str.rstrip('.wav')

    # 使用 str.split 拆分 'path' 欄位
    df_test[['session', 'role', 'number', 'start_time', 'end_time']] = df_test['path'].str.split('_', expand=True)

    # 如果 'number' 欄位的末尾包含 '.wav'，進行一次額外的拆分
    df_test['number'] = df_test['number'].str.rstrip('.wav')

    # 將 'start_time' 和 'end_time' 欄位轉換為數值型別
    df_test[['start_time', 'end_time']] = df_test[['start_time', 'end_time']].astype(int)

    return df_test
df = Augment_info_df(df)


# Packer
def Packer(df_test) -> dict:
    People_dict = dict(tuple(df_test.groupby('session')))
    return People_dict

def Dialogueturn2corpus(data_frame, mode='text'): #mode can be 'pred_str' or 'text'
    # 按 'start_time' 列進行排序
    sorted_data_frame = data_frame.sort_values(by='start_time')

    # 添加前綴並使用 '\n' 進行拼接
    processed_text = sorted_data_frame.apply(lambda row: f"{row['role']}: {row[mode]}", axis=1).str.cat(sep='\n')

    return processed_text

def filter_people_dict(People_dict, mode="INV+PAR", verbose=False) -> dict:# 'INV' , 'PAR', 'INV+PAR'
    filtered_people_dict = {}

    for session, data_frame in People_dict.items():
        # 使用 query 過濾 'role' 為 'INV' 或 'PAR'
        if mode == "PAR":
            filtered_data_frame = data_frame.query("role == 'PAR'")
            filtered_people_dict[session] = filtered_data_frame
        elif mode == "INV":
            filtered_data_frame = data_frame.query("role == 'INV'")
            filtered_people_dict[session] = filtered_data_frame
        elif mode == "INV+PAR":
            filtered_people_dict[session] = data_frame
        else:
            raise OSError
    
    if verbose:
        # 印出過濾後的 People_dict 中每個 session 的 DataFrame
        for session, data_frame in filtered_people_dict.items():
            print(f"Session: {session}")
            print(data_frame)
            print("\n")

    return filtered_people_dict

# Dialogue Formatter
def Dialogue_Formatter(People_dict, sep="\n",role_mode='PAR')->dict:
    session_df=pd.DataFrame()
    for session, data_frame in People_dict.items():
        if len(data_frame)>0:
            total_info=data_frame.iloc[0].copy()
            sessional_text = Dialogueturn2corpus(data_frame,mode='text')
            sessional_predStr = Dialogueturn2corpus(data_frame,mode='pred_str')
            
            # total_info,'text']=sessional_text
            # session_df.loc[session,'pred_str']=sessional_predStr
            # session_df.loc[session,'role']=role_mode
            # session_df.loc[session,'start_time']=data_frame['start_time'].min()
            # session_df.loc[session,'end_time']=data_frame['end_time'].max()

            total_info['text']=sessional_text
            total_info['pred_str']=sessional_predStr
            total_info['role']=role_mode
            total_info['start_time']=data_frame['start_time'].min()
            total_info['end_time']=data_frame['end_time'].max()
            session_df = pd.concat([session_df, pd.DataFrame([total_info], index=[session])])
        else:
            print(f"Session {session} has no data")
    return session_df
df_train = df[df['ex'] == 'train']
df_val = df[df['ex'] == 'dev']
df_test = df[df['ex'] == 'test']

def SentenceLvldf2SessionLvldf(df, role_mode="PAR"):
    People_dict=Packer(df)
    People_dict = filter_people_dict(People_dict, mode=role_mode, verbose=False)
    df_dialogue=Dialogue_Formatter(People_dict,role_mode)
    return df_dialogue




df_train=SentenceLvldf2SessionLvldf(df_train)
df_val=SentenceLvldf2SessionLvldf(df_val)
df_test=SentenceLvldf2SessionLvldf(df_test)

def Tokenize(df_data):
    df_data[t_col_name] = df_data[t_col_name].map(lambda x: self.t_tokenizer.encode(
        str(x),
        padding = 'max_length',
        max_length=self.args.max_length,
        truncation=True,
        ))
    return df_data
df_train=Tokenize(df_train)
df_val=Tokenize(df_val)
df_test=Tokenize(df_test)
df_test = df_test.reset_index(drop=True)

# audio_root="/mnt/Internal/FedASR/Data/ADReSS-IS2020-data/clips"
# # 원래 길이: 562992, batch 16: 90000, batch 8: 140000
# # max_length=16000, truncation=True 이건 일단 돌려보고 결정 => 뒤쪽, 앞에쪽 뭐보면 좋을 지 그런거 check하면 좋으니까! 
# df[a_col_name] = df[a_col_name].map(lambda x: self.a_tokenizer(
#     f"{audio_root}/{x}",
#     sampling_rate = tg_sr,
#     max_length=100000, 
#     truncation=True
#     )['input_values'][0])



print(f'# of train:{len(df_train)}, val:{len(df_val)}, test:{len(df_test)}')


self.train_data = TensorDataset(
    torch.tensor(df_train[t_col_name].tolist(), dtype=torch.long),
    # torch.tensor(df_train[a_col_name].tolist(), dtype=torch.float),
    torch.tensor(df_train[self.label_cols].tolist(), dtype=torch.long),
)

self.val_data = TensorDataset(
        torch.tensor(df_val[t_col_name].tolist(), dtype=torch.long),
    #  torch.tensor(df_val[a_col_name].tolist(), dtype=torch.float),
    torch.tensor(df_val[self.label_cols].tolist(), dtype=torch.long),
)

self.test_data = TensorDataset(
        torch.tensor(df_test[t_col_name].tolist(), dtype=torch.long),
    #  torch.tensor(df_test[a_col_name].tolist(), dtype=torch.float),
    torch.tensor(df_test[self.label_cols].tolist(), dtype=torch.long),
        torch.tensor(df_test.index.tolist(), dtype=torch.long),
)


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-english and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Session S009 has no data
Session S011 has no data
Session S017 has no data
Session S027 has no data
Session S029 has no data
Session S052 has no data
Session S080 has no data
Session S084 has no data
Session S086 has no data
Session S118 has no data
Session S124 has no data
Session S138 has no data
Session S145 has no data
# of train:108, val:73, test:48
